# Pré processa dados 

In [27]:
import matplotlib.pyplot as plt
import numpy as np
import pandas as pd
import notas

In [28]:
# %load const.py
CO_ELEVA = 33178860
CO_PARQUE = 33065837
CO_SAO_BENTO = 33062633
CO_SAO_VICENTE = 33063648
CO_SAO_VICENTE = 33063648
CO_ST_INACIO = 33063729
CO_STO_AGO = 33020566
CO_STO_AGO_INST = 33176825
CO_STO_AGOS_NL = 33087679


CO_MUN = 3304557  # Rio de Janeiro
CO_UF_RIO = 33  # RJ

MIN_ALUNOS = 30  # Mínimo de alunos fazendo Enem para considerar na análise
NUM_MELHORES = 60
ANO_ULT = 2019  # último ano a processar

notas_cols = [
    "NU_NOTA_CN",
    "NU_NOTA_CH",
    "NU_NOTA_LC",
    "NU_NOTA_MT",
    "NU_NOTA_REDACAO",
]

In [3]:
# para poder atualizar em um só lugar
from const import ANO_ULT

## Lendo dados do Enem do Rio de Janeiro

In [32]:
from pathlib import Path


def encontra_arq_enem(ano):

    arq = Path(f"dados/microdados_enem{ano}/DADOS/MICRODADOS_ENEM_{ano}.csv")
    if not arq.exists():  # 2017
        arq = Path(
            f"dados/microdados_enem{ano}/Microdados Enem {ano}/DADOS/MICRODADOS_ENEM_{ano}.csv"
        )
    if not arq.exists():  # 2016
        arq = Path(
            f"dados/microdados_enem{ano}/Microdados_enem_{ano}/DADOS/MICRODADOS_ENEM_{ano}.csv"
        )

    if not arq.exists():  # 2012
        arq = Path(f"dados/microdados_enem{ano}/DADOS/DADOS_ENEM_{ano}.csv")
    assert arq.exists(), f"A porra do Inep não padronizou nome de enem para ano {ano}"
    return arq


# rar em 2013
for ano in range(2012, 2020):
    print(encontra_arq_enem(ano))


def read_enem_rio(ano=ANO_ULT):
    arq = encontra_arq_enem(ano)

    print("Lendo dados de:", arq)
    df_enem_rio = pd.concat(
        df_enem.rename(
            columns={
                "COD_ESCOLA": "CO_ESCOLA",
                "COD_UF_ESC": "CO_UF_ESC",
                "NOTA_CN": "NU_NOTA_CN",
                "NOTA_CH": "NU_NOTA_CH",
                "NOTA_LC": "NU_NOTA_LC",
                "NOTA_MT": "NU_NOTA_MT",
                'ST_CONCLUSAO': 'TP_ST_CONCLUSAO',
                'IN_TP_ENSINO': 'TP_ENSINO',
            }
        ).query('CO_UF_ESC == @CO_UF_RIO')
        for df_enem in pd.read_csv(
            arq,
            chunksize=50000,
            sep="," if ano in (2015, 2014, 2012) else ";",
            encoding="iso8859-1",
        )
    )
    df_enem_rio.dropna(subset=["CO_ESCOLA"], inplace=True)
    df_enem_rio["CO_ESCOLA"] = df_enem_rio.CO_ESCOLA.astype(int)
    return df_enem_rio

dados/microdados_enem2012/DADOS/DADOS_ENEM_2012.csv
dados/microdados_enem2013/Microdados_enem_2013/DADOS/MICRODADOS_ENEM_2013.csv
dados/microdados_enem2014/DADOS/MICRODADOS_ENEM_2014.csv
dados/microdados_enem2015/DADOS/MICRODADOS_ENEM_2015.csv
dados/microdados_enem2016/Microdados_enem_2016/DADOS/MICRODADOS_ENEM_2016.csv
dados/microdados_enem2017/Microdados Enem 2017/DADOS/MICRODADOS_ENEM_2017.csv
dados/microdados_enem2018/DADOS/MICRODADOS_ENEM_2018.csv
dados/microdados_enem2019/DADOS/MICRODADOS_ENEM_2019.csv


In [25]:
%reset


Once deleted, variables cannot be recovered. Proceed (y/[n])?  y


dados/microdados_enem2014/DADOS/MICRODADOS_ENEM_2014.csv
dados/microdados_enem2015/DADOS/MICRODADOS_ENEM_2015.csv


AssertionError: A porra do Inep não padronizou nome de enem para ano 2016

In [29]:
df_enem_rio = read_enem_rio()
df_enem_rio.head()

,NU_INSCRICAO,NU_ANO,CO_MUNICIPIO_RESIDENCIA,NO_MUNICIPIO_RESIDENCIA,CO_UF_RESIDENCIA,SG_UF_RESIDENCIA,NU_IDADE,TP_SEXO,TP_ESTADO_CIVIL,TP_COR_RACA,...,Q016,Q017,Q018,Q019,Q020,Q021,Q022,Q023,Q024,Q025
144,190001517230,2019,3300100,Angra dos Reis,33,RJ,18.0,F,1,3,...,B,A,B,D,B,A,D,A,A,B
251,190001517237,2019,3305802,Teresópolis,33,RJ,18.0,F,1,0,...,B,A,A,B,A,A,C,A,B,B
323,190001520685,2019,3303906,Petrópolis,33,RJ,20.0,F,1,1,...,A,A,A,A,B,A,C,A,B,A
423,190001520689,2019,3301702,Duque de Caxias,33,RJ,22.0,F,1,3,...,A,A,A,B,A,A,B,B,A,A
487,190001520694,2019,3302106,Itaocara,33,RJ,17.0,M,1,1,...,A,A,A,C,B,A,D,A,B,B


In [5]:
df_enem_rio.shape

(73086, 136)

In [7]:
df_sv = df_enem_rio.loc[df_enem_rio.CO_ESCOLA == CO_SAO_VICENTE]
notas = list(filter(lambda x: "NOTA" in x, df_sv.columns.to_list()))
df_sv[notas].mean()

NU_NOTA_CN         582.858065
NU_NOTA_CH         622.669355
NU_NOTA_LC         608.045161
NU_NOTA_MT         708.056452
NU_NOTA_COMP1      154.516129
NU_NOTA_COMP2      168.387097
NU_NOTA_COMP3      163.870968
NU_NOTA_COMP4      169.677419
NU_NOTA_COMP5      161.290323
NU_NOTA_REDACAO    817.741935
dtype: float64

## Lendo censo de escolas

Alguns dados que parecem interessantes:

**TP_SITUACAO_FUNCIONAMENTO**	Situação de funcionamento	Num	1	1 - Em Atividade 2 - Paralisada 3 - Extinta (ano do Censo) 4 - Extinta em Anos Anteriores

TP_DEPENDENCIA	Dependência Administrativa	Num	1	1 - Federal 2 - Estadual 3 - Municipal 4 - Privada

TP_CATEGORIA_ESCOLA_PRIVADA	Categoria da escola privada	Num	1	1 - Particular 2 - Comunitária 3 - Confessional 4 - Filantrópica    - Não aplicável para escolas públicas

IN_MANT_ESCOLA_PRIVADA_EMP	Mantenedora da escola privada - Empresa, grupo empresarial do setor privado ou pessoa física

IN_MANT_ESCOLA_PRIVADA_ONG	Mantenedora da escola privada - Organização não governamental (ONG) - internacional ou nacional. Organização da Sociedade Civil de Interesse Público (Oscip)

IN_MANT_ESCOLA_PRIVADA_SIND	Mantenedora da escola privada - Sindicatos de trabalhadores ou patronais, associações e cooperativas

IN_MANT_ESCOLA_PRIVADA_SIST_S	Mantenedora da escola privada - Sistema S (Sesi, Senai, Sesc, Outros)

IN_MANT_ESCOLA_PRIVADA_S_FINS	Mantenedora da escola privada - Instituições sem fins lucrativos

QT_SALAS_EXISTENTES	Número de salas de aula existentes na escola	Num	4

QT_SALAS_UTILIZADAS	Número de salas utilizadas como salas de aula (dentro e fora do prédio)	Num	4

QT_FUNCIONARIOS	Total de funcionários da escola (inclusive profissionais escolares em sala de aula)

**IN_COMUM_MEDIO_MEDIO**	Escola oferece Ensino Médio - Médio Propedêutico em classes comuns do ensino regular (TP_ETAPA_ENSINO igual a 25,26,27,28 ou 29 e IN_ESPECIAL_EXCLUSIVA=0)	Num	1	0 - Não 1 - Sim


In [6]:
df_escolas = pd.read_csv(
    f"dados/microdados_educacao_basica_{ANO_ULT}/microdados_educacao_basica_{ANO_ULT}/DADOS/ESCOLAS.CSV",
    sep="|",
    encoding="latin1",
    low_memory=False,
    index_col="CO_ENTIDADE",
).query(
    "CO_UF == @CO_UF_RIO and IN_COMUM_MEDIO_MEDIO==1 and TP_SITUACAO_FUNCIONAMENTO==1"
)
df_escolas

,NU_ANO_CENSO,NO_ENTIDADE,CO_ORGAO_REGIONAL,TP_SITUACAO_FUNCIONAMENTO,DT_ANO_LETIVO_INICIO,DT_ANO_LETIVO_TERMINO,CO_REGIAO,CO_MESORREGIAO,CO_MICRORREGIAO,CO_UF,...,IN_ESP_EXCLUSIVA_MEDIO_INTEGR,IN_ESP_EXCLUSIVA_MEDIO_NORMAL,IN_COMUM_EJA_FUND,IN_COMUM_EJA_MEDIO,IN_COMUM_EJA_PROF,IN_ESP_EXCLUSIVA_EJA_FUND,IN_ESP_EXCLUSIVA_EJA_MEDIO,IN_ESP_EXCLUSIVA_EJA_PROF,IN_COMUM_PROF,IN_ESP_EXCLUSIVA_PROF
CO_ENTIDADE,,,,,,,,,,,,,,,,,,,,,
33117306,2019,COLEGIO ATENAS,11006,1,04/02/2019,12/12/2019,3,3306,33018,33,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
33074089,2019,CENTRO EDUCACIONAL ROSA CHAMMA,11006,1,04/02/2019,20/12/2019,3,3306,33018,33,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
33074151,2019,CE MATO GROSSO,11006,1,01/01/2019,31/12/2019,3,3306,33018,33,...,0.0,0.0,0.0,1.0,0.0,0.0,0.0,0.0,0.0,0.0
33074267,2019,CE RODRIGO OTAVIO FILHO,11006,1,01/01/2019,31/12/2019,3,3306,33018,33,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
33075123,2019,SOCIEDADE EDUC VILA DA PENHA,11006,1,04/02/2019,30/12/2019,3,3306,33018,33,...,0.0,0.0,0.0,1.0,0.0,0.0,0.0,0.0,0.0,0.0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
33106657,2019,CIEP 183 JOAO VITTA,11007,1,01/01/2019,31/12/2019,3,3306,33018,33,...,0.0,0.0,0.0,1.0,0.0,0.0,0.0,0.0,0.0,0.0
33107203,2019,SOCIEDADE EDUCACIONAL RAMOS PINTO,11009,1,06/02/2019,20/12/2019,3,3306,33018,33,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
33108412,2019,JD ESC CARA MELADA LTDA ME - CENTRO EDUC ARGOS,11007,1,04/02/2019,20/12/2019,3,3306,33018,33,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0


## Lê dados históricos de turmas 

In [8]:
primeiro_ano = [25, 30, 35]

In [9]:
import glob
import os.path

import patoolib


def dados_turma(ano, try_rar=True):
    dir_censo = [
        i
        for i in glob.glob(f"dados/*{ano}*")
        if ("censo" in i or "educacao_basica" in i) and "zip" not in i
    ]
    assert (
        len(dir_censo) == 1
    ), f"Só pode ter achado um arquivo e achou {len(dir_censo)}"

    arquivo_turmas = glob.glob(f"{dir_censo[0]}/*{ano}*/DADOS/TURMAS.*")
    if not arquivo_turmas:
        arquivo_turmas = glob.glob(f"{dir_censo[0]}/DADOS/TURMAS.*")

    result = [i for i in arquivo_turmas if i.lower().endswith(".csv")]
    if not result:
        result = [i for i in arquivo_turmas if i.lower().endswith(".zip")]
        if try_rar and not result:
            rar = [i for i in arquivo_turmas if i.lower().endswith(".rar")]
            if rar:
                patoolib.extract_archive(rar[0], outdir=os.path.dirname(rar[0]))
                result = dados_turma(ano, False)

    if result:
        result = result[0]
    else:
        result = None
    return result


for i in range(2007, ANO_ULT + 1):
    print(">", dados_turma(i))
# estrutura diretórios bagunçados anets de 2007

> dados/micro_censo_escolar_2007/2007/DADOS/TURMAS.CSV
> dados/micro_censo_escolar_2008/2008/DADOS/TURMAS.CSV
> dados/micro_censo_escolar_2009/2009/DADOS/TURMAS.CSV
> dados/micro_censo_escolar_2010/2010/DADOS/TURMAS.CSV
> dados/micro_censo_escolar_2011/2011/DADOS/TURMAS.CSV
> dados/micro_censo_escolar_2012/2012/DADOS/TURMAS.CSV
> dados/micro_censo_escolar_2013/2013/DADOS/TURMAS.CSV
> dados/micro_censo_escolar_2014/2014/DADOS/TURMAS.CSV
> dados/micro_censo_escolar_2015/2015/DADOS/TURMAS.CSV
> dados/micro_censo_escolar_2016/micro_censo_escolar_2016/DADOS/TURMAS.CSV
> dados/micro_censo_escolar_2017/Microdados_Censo_Escolar_2017/DADOS/TURMAS.zip
> dados/microdados_educacao_basica_2018/microdados_ed_basica_2018/DADOS/TURMAS.CSV
> dados/microdados_educacao_basica_2019/microdados_educacao_basica_2019/DADOS/TURMAS.CSV


In [10]:
col_turmas = pd.DataFrame(
    {
        "id_etapa": [
            "category",
            "FK_COD_ETAPA_ENSINO",
            "TP_ETAPA_ENSINO",
            "TP_ETAPA_ENSINO",
        ],
        "id_escola": [
            "category",
            "PK_COD_ENTIDADE",
            "CO_ENTIDADE",
            "CO_ENTIDADE",
        ],
        "num_matriculas": [
            "uint8",
            "NUM_MATRICULAS",
            "NU_MATRICULAS",
            "QT_MATRICULAS",
        ],
        "id_municipio": [
            "category",
            "FK_COD_MUNICIPIO",
            "CO_MUNICIPIO",
            "CO_MUNICIPIO",
        ],
        "id_uf": [
            "category",
            "FK_COD_ESTADO",
            "CO_UF",
            "CO_UF",
        ],
        #'is_eja':         ['boolean',  'IN_EJA',               'IN_EJA',          'IN_EJA',          ],
    },
    index=[
        "dtype",
        "2007",
        "2015",
        "2018",
    ],
)


def colunas_turmas(columns):
    columns = set(columns)
    match = col_turmas.applymap(columns.__contains__).all(axis=1)
    assert match.sum() == 1, (
        f"Um e somente um registro de nomes de colunas deveria "
        f"casar com todas as colunas dos dados lidos de disco {str(columns)}\n"
        f"\n{match}"
    )
    colunas_map = col_turmas[match].iloc[0].to_dict()
    return colunas_map

In [11]:
def mapa_de_colunas(ano):
    df_teste = pd.read_csv(dados_turma(ano), sep="|", encoding="latin1", nrows=1)
    map_colunas = colunas_turmas(df_teste.columns)

    return {v: k for k, v in map_colunas.items()}


# mapa_de_colunas(2014), mapa_de_colunas(2015)

In [12]:
def le_turma(ano):
    print(f"Lendo ano {ano}")
    map_colunas = mapa_de_colunas(ano)
    df_t = pd.concat(
        (
            df_t  # .rename(columns=map_colunas).query('id_etapa.isin(@primeiro_ano) and id_uf == @CO_MUN_RIO')\
            # .merge(escolas, left_on='id_escola', right_on='CO_ENTIDADE')
            for df_t in pd.read_csv(
                dados_turma(ano),
                sep="|",
                encoding="latin1",
                chunksize=1000,
                # error_bad_lines=False,
                usecols=map_colunas.keys(),
                # nrows=100 #debug
            )
        ),
        sort=True,
    )
    df_t["ano"] = ano
    return df_t


# df_primeiro_ano_turmas = pd.concat(
#     (le_turma(i, df_escolas.CO_ENTIDADE, primeiro_ano) for i in range(2007, 2019)), #2007 e 2008, 2009 falharam linhas diferentes
#     sort=True)
# print(df_primeiro_ano_turmas.shape)
# df_primeiro_ano_turmas.head()
def le_tudo():
    for i in range(2015, ANO_ULT + 1):
        print(f"Turmas ano {i}")
        le_turma(i)


le_tudo()

Turmas ano 2015
Lendo ano 2015
Turmas ano 2016
Lendo ano 2016
Turmas ano 2017
Lendo ano 2017
Turmas ano 2018
Lendo ano 2018
Turmas ano 2019
Lendo ano 2019


In [23]:
def le_turmas_padronizadas(filtro, range_=None):
    if not range_:
        range_ = range(2007, ANO_ULT+1)

    return pd.concat(
        [
            (le_turma(i)
            .rename(
                columns=mapa_de_colunas(i)
            )            .astype(col_turmas.loc['dtype'].to_dict()) #precisa do to_dict?
            .query(filtro)
            .assign(ano=i))
            for i in range_
        ],
        sort=True,
    )

In [24]:
df_primeiro_ano_turmas = le_turmas_padronizadas(
    "id_etapa.isin(@primeiro_ano) and id_uf == @CO_UF_RIO"
)
print(df_primeiro_ano_turmas.shape)
df_primeiro_ano_turmas.head()

Lendo ano 2007
Lendo ano 2008
Lendo ano 2009
Lendo ano 2010
Lendo ano 2011
Lendo ano 2012
Lendo ano 2013
Lendo ano 2014
Lendo ano 2015
Lendo ano 2016
Lendo ano 2017
Lendo ano 2018
Lendo ano 2019
(100182, 6)


,ano,id_escola,id_etapa,id_municipio,id_uf,num_matriculas
6930,2007,33015279,25.0,3302403,33,30
6952,2007,33015279,25.0,3302403,33,32
7044,2007,33055653,25.0,3303302,33,37
7045,2007,33055653,25.0,3303302,33,34
7046,2007,33055653,25.0,3303302,33,34


## Grava tudo em formato binário em disco

In [17]:
# df_turmas.reset_index().to_feather('dados/turmas2018.feather')
df_enem_rio.reset_index().to_feather(f"dados/enem_rio_{ANO_ULT}.feather")

In [28]:
df_escolas.reset_index().to_feather(f"dados/escolas_rio_{ANO_ULT}.feather")

In [25]:
df_primeiro_ano_turmas.reset_index().to_feather("dados/primeiro_ano.feather")

In [26]:
df_turmas_rio = le_turmas_padronizadas("id_uf == @CO_UF_RIO")

Lendo ano 2007
Lendo ano 2008
Lendo ano 2009
Lendo ano 2010
Lendo ano 2011
Lendo ano 2012
Lendo ano 2013
Lendo ano 2014
Lendo ano 2015
Lendo ano 2016
Lendo ano 2017
Lendo ano 2018
Lendo ano 2019


In [27]:
df_turmas_rio.reset_index().to_feather("dados/turmas_rio.feather")

## Grava notas de todos os anos do Enem
Enem no formato atual começou mesmo foi em 2009. Ignorarei outros anos.

In [3]:
df_enem_rio = pd.read_feather(f"dados/enem_rio_2019.feather")

In [4]:
import notas

notas.notas_enem(df_enem_rio)

,CO_ESCOLA,NU_ANO,NU_NOTA_CN,NU_NOTA_CH,NU_NOTA_LC,NU_NOTA_MT,NU_NOTA_REDACAO,num_alunos,mediana,perc_80,rank
0,33135827,2019,647.50,651.00,627.00,789.00,940.0,31.0,725.0,746.0,1
1,33062633,2019,636.50,672.00,626.00,789.50,900.0,45.0,722.0,743.0,2
2,33178860,2019,610.00,662.50,626.00,779.00,910.0,60.0,711.5,744.0,3
3,33176825,2019,605.50,649.00,620.50,759.00,920.0,146.0,707.0,735.0,4
4,33062609,2019,611.50,649.50,612.50,777.50,880.0,80.0,707.0,738.0,4
...,...,...,...,...,...,...,...,...,...,...,...
538,33149119,2019,449.25,451.50,508.00,437.50,500.0,40.0,467.5,522.5,539
539,33088152,2019,447.00,496.25,507.00,438.50,520.0,33.0,465.0,527.0,540
540,33008485,2019,425.75,460.50,475.25,454.25,540.0,39.0,464.0,517.0,541
541,33041946,2019,421.00,445.00,501.00,501.00,500.0,33.0,463.0,550.0,542


df_notas_todos_enem = pd.concat(
    [
        notas.notas_enem(pd.read_feather(f"dados/enem_rio_{ano}.feather"))
        for ano in range(2009, ANO_ULT + 1)
    ]
)



In [28]:
df = read_enem_rio(2017)

In [29]:
df.NU_ANO

144        2019
251        2019
323        2019
423        2019
487        2019
           ... 
5093701    2019
5094154    2019
5094208    2019
5094329    2019
5094349    2019
Name: NU_ANO, Length: 73086, dtype: int64

In [23]:
df_enem_tudo= []
for ano in range(2009, ANO_ULT + 1):
    print(f"{ano=}")
    df = read_enem_rio(ano)
    print("... calcula notas")
    dfs.append(notas.notas_enem(df))
print("concatenando")

In [27]:
dfs[0].NU_ANO.value_counts()

2019    543
Name: NU_ANO, dtype: int64

In [45]:
%pdb off
pd.concat(
    notas.notas_enem(read_enem_rio(ano))
    for ano in range(
        ANO_ULT,
         2013 - 1, -1
    )  # enem começa pra valer em 2009, mas dados de 2011 vêm na porra de um formato diferente de todos os outros. E 2012 tem colunas também completamente diferentes
).reset_index(drop=True).to_feather("dados/enem_anos.feather")

Automatic pdb calling has been turned OFF
Lendo dados de: dados/microdados_enem2019/DADOS/MICRODADOS_ENEM_2019.csv
Lendo dados de: dados/microdados_enem2018/DADOS/MICRODADOS_ENEM_2018.csv
Lendo dados de: dados/microdados_enem2017/Microdados Enem 2017/DADOS/MICRODADOS_ENEM_2017.csv
Lendo dados de: dados/microdados_enem2016/Microdados_enem_2016/DADOS/MICRODADOS_ENEM_2016.csv
Lendo dados de: dados/microdados_enem2015/DADOS/MICRODADOS_ENEM_2015.csv
Lendo dados de: dados/microdados_enem2014/DADOS/MICRODADOS_ENEM_2014.csv
Lendo dados de: dados/microdados_enem2013/Microdados_enem_2013/DADOS/MICRODADOS_ENEM_2013.csv


In [43]:
%pdb on
notas.notas_enem(read_enem_rio(2019))

Automatic pdb calling has been turned ON
Lendo dados de: dados/microdados_enem2019/DADOS/MICRODADOS_ENEM_2019.csv


,CO_ESCOLA,Ano,nota_ciências,nota_humanas,nota_português,nota_matemática,nota_redação,nota_final,num_alunos,nota_ciências_perc80,nota_humanas_perc80,nota_português_perc80,nota_matemática_perc80,nota_redação_perc80,nota_final_perc80,rank
0,33135827,2019,647.50,651.00,627.00,789.00,940.0,725.0,31.0,676.00,685.50,641.50,823.00,960.0,746.0,1
1,33062633,2019,636.50,672.00,626.00,789.50,900.0,722.0,45.0,662.50,714.00,641.50,853.00,940.0,743.0,2
2,33178860,2019,610.00,662.50,626.00,779.00,910.0,711.5,60.0,652.50,683.50,648.00,848.00,940.0,744.0,3
3,33176825,2019,605.50,649.00,620.50,759.00,920.0,707.0,146.0,643.00,682.00,644.50,815.50,960.0,735.0,4
4,33062609,2019,611.50,649.50,612.50,777.50,880.0,707.0,80.0,649.50,682.00,632.00,831.00,920.0,738.0,4
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
551,33149119,2019,444.50,434.00,507.50,437.50,480.0,460.0,45.0,487.50,514.00,537.50,502.00,600.0,515.5,552
552,33008485,2019,422.25,452.25,467.25,454.25,540.0,459.0,42.0,487.00,514.00,513.00,501.75,600.0,514.0,553
553,33049726,2019,428.75,455.25,485.50,457.50,420.0,457.0,35.0,473.75,520.50,538.50,505.25,604.0,514.0,554
554,33061874,2019,417.25,439.75,475.75,432.00,520.0,451.0,44.0,478.25,486.75,511.25,497.00,580.0,485.5,555
